#**خدوم**

In [3]:
pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.0 MB/s eta 0:00:00


In [3]:
%%writefile app.py

import streamlit as st
from huggingface_hub import InferenceClient
import PyPDF2
from openai import OpenAI
import re
import base64

client = OpenAI(
    base_url="https://huggingface.co/api/inference-proxy/together",
    api_key=""  # Replace with your own key
)
# ---------------- UI -----------------------------------------
st.set_page_config(page_title="Qwen Chatbot", layout="centered")

st.title("ServeItUp")
st.image("/content/IMG_9054 (1).PNG", width=100)  # Adjust width as needed

st.write("Chat with AI customer Service")
st.write("Upload a PDF (FAQs, company policies)")


#---------------------------------------------------------------------


uploaded_file = st.file_uploader("Upload a PDF document", type="pdf")

if "document_text" not in st.session_state:
    st.session_state.document_text = ""

if uploaded_file:
    pdf_reader = PyPDF2.PdfReader(uploaded_file)
    extracted_text = "\n".join([page.extract_text() for page in pdf_reader.pages if page.extract_text()])
    st.session_state.document_text = extracted_text
    st.success("✅ PDF uploaded and processed!")

# ------------------ System Prompt for Chatbot ------------------
SYSTEM_PROMPT = """You are an AI customer support assistant.
You MUST answer concisely based ONLY on the provided document.

### Rules:
- **Only answer from the document**.
- **DO NOT speculate or make up answers**.
- **If the answer is unknown, respond:** "I don't have that information in the document."
- **DO NOT repeat or rephrase the user’s question**.
- **DO NOT start responses with 'User:' or repeat conversation history**.
- **DO NOT include "<think>" statements or reasoning processes**.
- **DO NOT ignore polite feedback like 'That was helpful' or 'Thank you'—instead, respond with 'You're welcome.'**
- **Give direct answers in a professional tone**.

### Examples:
**User:** Who wrote 'Hamlet'?
**Assistant:** William Shakespeare.

**User:** That was helpful.
**Assistant:** You're welcome.

**User**: This is not what I asked for.
**Assistant**: I’m sorry, let me clarify.

**User**: You're not answering my question correctly.
**Assistant**: I apologize for any confusion, I am under development, your question will be handled by a human.

**User:** What is the CEO’s name?
**Assistant:** I don’t have that information in the document.
"""

if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": SYSTEM_PROMPT}
    ]

for message in st.session_state.messages:
    if message["role"] == "system":
        continue
    role = "🤖" if message["role"] == "assistant" else "👤"
    st.chat_message(role).write(message["content"])

# ------------------ User Chat Input ------------------
user_input = st.chat_input("Type your message...")

if user_input and st.session_state.document_text:
    st.session_state.messages.append({"role": "user", "content": user_input})

    completion = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1",
        messages=st.session_state.messages,
        max_tokens=256
    )

    bot_reply = completion.choices[0].message.content.strip()
    bot_reply = re.sub(r"<think>.*?</think>", "", bot_reply, flags=re.DOTALL).strip()

    st.session_state.messages.append({"role": "assistant", "content": bot_reply})

    st.chat_message("👤").write(user_input)
    st.chat_message("🤖").write(bot_reply)

elif user_input:
    st.warning("⚠️ Please upload a PDF first!")

# ------------------ Sentiment Analysis Prompt ------------------
SENTIMENT_PROMPT = """Analyze the conversation and determine the customer's satisfaction level.

### **Rules:**
- **A normal Q&A conversation where the AI provides an answer → "The customer is satisfied."**
- **If the user complains, expresses frustration, or says the assistant is wrong or 'I am not happy' → "The customer is not satisfied."**
- **If the user expresses gratitude (e.g., "Thanks," "That was helpful") → "The customer is satisfied."**
- **ONLY return one of these two responses:**
  - "The customer is satisfied."
  - "The customer is not satisfied."
- **DO NOT explain your reasoning.**
- **DO NOT add any extra text or "<think>" statements.**
- **DO NOT assume dissatisfaction unless the user explicitly expresses frustration.**

### **Examples:**
**Example 1 (Satisfied)**
User: Who wrote 'Hamlet'?
Assistant: William Shakespeare.
User: That was helpful.
Assistant: You're welcome.
Output → "The customer is satisfied."

**Example 2 (Not Satisfied)**
User: You're not answering my question correctly.
Assistant: I apologize for any confusion, I am under development, your question will be handled by a human.
Output → "The customer is not satisfied."

**Example 3 (User says "I am not happy" - Not Satisfied)**
User: This is not what I asked for, I am not happy with your answer.
Assistant: I’m sorry, let me clarify.
Output → "The customer is not satisfied."

**Example 4 (Satisfied, Neutral but Helpful)**
User: Thank you, that was helpful!
Assistant: You're welcome!
Output → "The customer is satisfied."

**Example 5 (Frustrated User, Not Satisfied)**
User: This is not what I asked for.
Assistant: I’m sorry, let me clarify.
Output → "The customer is not satisfied."

**Example 6 (Short Conversation, But Answered Correctly)**
User: What is the capital of France?
Assistant: Paris.
Output → "The customer is satisfied."

Now analyze the following conversation and determine the final result:
"""

# ------------------ "End Conversation" for Sentiment ------------------
if st.button("End Conversation"):
    if len(st.session_state.messages) > 1:
        formatted_conversation = "\n".join([
            f"{msg['role'].capitalize()}: {msg['content']}"
            for msg in st.session_state.messages if msg["role"] != "system"
        ])

        sentiment_response = client.chat.completions.create(
            model="deepseek-ai/DeepSeek-R1",
            messages=[
                {"role": "system", "content": "You are a sentiment analysis assistant. Your task is to analyze customer satisfaction based on conversation history."},
                {"role": "user", "content": SENTIMENT_PROMPT + formatted_conversation}
            ],
            max_tokens=50
        )

        sentiment_result = sentiment_response.choices[0].message.content.strip()
        sentiment_result = re.sub(r"<think>.*?</think>", "", sentiment_result, flags=re.DOTALL).strip()  # Ensure <think> is removed


        conversation_lower = formatted_conversation.lower()
        NEGATIVE_KEYWORDS = [
            "this is not what i asked for",
            "i am not happy",
            "not answering my question correctly",
            "upset",
            "angry",
            "frustrated"
        ]
        if any(keyword in conversation_lower for keyword in NEGATIVE_KEYWORDS):
            final_sentiment = "The customer is not satisfied."
        else:
            if "not satisfied" in sentiment_result:
                final_sentiment = "The customer is not satisfied."
            else:
                final_sentiment = "The customer is satisfied."

        st.subheader("Conversation Sentiment Analysis:")
        st.success(final_sentiment)

        st.session_state.messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    else:
        st.warning("⚠️ No conversation available for sentiment analysis!")


Overwriting app.py


In [7]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [5]:
pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.0 MB/s eta 0:00:00


In [4]:
import ngrok
secret = "2rrXTthXeGOp6AHKIwlIHdIRyUj_7Qss4a5AkTeBJViSsajJH"
# asynchronous execution: do this but do not wait for it until it is done, complete and go run the next line
listner = await ngrok.forward(8501, authtoken =secret)
print(listner.url())

!streamlit run app.py

https://a27e-34-125-7-90.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.7.90:8501

  Stopping...
